In [355]:
import cv2
import numpy as np

In [356]:
import cv2

img = cv2.imread('03_image.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#自適應直方圖等化
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(4,4))
eq = clahe.apply(img_gray)
cv2.imwrite('histogram_equalization.jpg', eq)

#中值濾波
denoised = cv2.medianBlur(eq, 5)
cv2.imwrite('denoised.jpg', denoised)


True

In [365]:
def motion_blur(kernel_size, angle):
    """
    對影像添加運動模糊效果，並顯示每一步驟
    :param image: 原始影像 (彩色或灰階)
    :param kernel_size: 模糊核大小
    :param angle: 模糊方向角度
    :return: 添加模糊後的影像
    """
    # 初始化模糊核
    kernel = np.zeros((kernel_size, kernel_size)) # 開一個全黑畫布
    center = kernel_size // 2 # 計算核的中心位置
    cv2.line(kernel, (center, 0), (center, kernel_size - 1), 1, thickness=1) # 在核中畫一條垂直的直線

    # 旋轉模糊核
    rotation_matrix = cv2.getRotationMatrix2D((center, center), angle, 1) # 生成旋轉矩陣
    kernel = cv2.warpAffine(kernel, rotation_matrix, (kernel_size, kernel_size)) # 將旋轉矩陣應用到模糊核上


    # 標準化模糊核
    kernel /= kernel.sum()


    return kernel
def wiener_filter(image, kernel, K=0.01):
    """
    使用維納濾波還原模糊影像，並顯示每一步驟
    :param image: 模糊影像 (彩色或灰階)
    :param kernel: 模糊核
    :param K: 噪聲功率比
    :return: 還原後的影像
    """
    result = np.zeros_like(image)
    if len(image.shape) == 2:  # 灰階影像
        kernel_padded = np.pad(kernel, [(0, image.shape[0] - kernel.shape[0]), 
                                        (0, image.shape[1] - kernel.shape[1])], 
                               mode='constant')

        kernel_fft = np.fft.fft2(kernel_padded)
        image_fft = np.fft.fft2(image)

        kernel_conj = np.conj(kernel_fft)
        wiener_result = (kernel_conj / (np.abs(kernel_fft)**2 + K)) * image_fft
        result = np.abs(np.fft.ifft2(wiener_result))
    else:  # 彩色影像
        for i in range(image.shape[2]):  # 對每個通道處理
            channel = image[:, :, i]
            kernel_padded = np.pad(kernel, [(0, channel.shape[0] - kernel.shape[0]), 
                                            (0, channel.shape[1] - kernel.shape[1])], 
                                   mode='constant')
            kernel_fft = np.fft.fft2(kernel_padded)
            channel_fft = np.fft.fft2(channel)
            kernel_conj = np.conj(kernel_fft)
            wiener_result = (kernel_conj / (np.abs(kernel_fft)**2 + K)) * channel_fft
            result[:, :, i] = np.abs(np.fft.ifft2(wiener_result))
    return result

# 模擬動態模糊
kernel_size = 100 # 模糊核大小
angle = 300  # 模糊方向角度
blur_kernel = motion_blur(kernel_size, angle)

# 還原模糊影像
restored_img = wiener_filter(denoised, blur_kernel, K=0.0158)

# 顯示還原影像
cv2.imwrite("result.jpg",restored_img)

True